# Heroes of Pymoli Data Analysis
--------------------------------

##Males make up a disproportionate number of total players, over 80%.
##Most players are 24 or younger, over 80%.
##The most popular item is not the most profitable.


In [1]:
import pandas as pd    
pd.options.display.float_format = '${:,.2f}'.format

#reading json file
file = 'purchase_data.json'
df=pd.read_json(file)
df.head(10)

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,$3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",$2.32,Eolo46
2,34,Male,174,Primitive Blade,$2.46,Assastnya25
3,21,Male,92,Final Critic,$1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,$1.27,Aela59
5,20,Male,10,Sleepwalker,$1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,$4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",$3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",$2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",$4.53,Hilaerin92


In [2]:
#Player Count
player_count = df.groupby(['SN']).size()
player_count.count()
print(player_count.count())

#output data frame
df_tot_players = pd.DataFrame({
    "Total Players": [player_count.count()]
})
df_tot_players

573


,Total Players
0,573


In [3]:
#Purchasing Analysis (Total)**
#Number of Unique Items
pd.options.display.float_format = '${:,.2f}'.format

group_by_unique_items = df.groupby(['Item Name']).size()

group_by_unique_items.count()

#Total Number of Purchases
len(df.index)

#output data frame
df_purchasing_analysis = pd.DataFrame({
    "Number of Unique Items": [group_by_unique_items.count()],
    "Average Purchase Price": df["Price"].mean(),
    "Total Number of Purchases": len(df.index),
    "Total Revenue": (df['Price'].sum())
})
df_purchasing_analysis
df_purchasing_analysis[["Number of Unique Items","Average Purchase Price","Total Number of Purchases","Total Revenue"]]

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$2.93,780,"$2,286.33"


In [4]:
#Gender Demographics
pd.options.display.float_format = '{:,.2f}'.format

#group_by_sn = df.loc["Female"].groupby(['Gender','SN']).size().reset_index()
only_females = df.loc[df["Gender"] == "Female", :].groupby('SN').size()

#count of each gender
female_count=len(df.loc[df["Gender"] == "Female", :].groupby('SN').size())
male_count=len(df.loc[df["Gender"] == "Male", :].groupby('SN').size())
other_count=len(df.loc[df["Gender"] == "Other / Non-Disclosed", :].groupby('SN').size())

#Percentage and Count of Male Players
male_players_percentage=len(df.loc[df["Gender"] == "Male", :].groupby('SN').size())/player_count.count()*100

#Percentage and Count of Female Players
female_players_percentage=len(df.loc[df["Gender"] == "Female", :].groupby('SN').size())/player_count.count()*100

#Percentage and Count of Other / Non-Disclosed
other_players_percentage=len(df.loc[df["Gender"] == "Other / Non-Disclosed", :].groupby('SN').size())/player_count.count()*100

df_gender_dem = pd.DataFrame({'Gender': ["Female","Male","Other / Non-Disclosed"], 
                   'Percentage of Players': [female_players_percentage,male_players_percentage,other_players_percentage],
                   'Total Count': [female_count,male_count,other_count]
                             }).set_index('Gender')
df_gender_dem

,Percentage of Players,Total Count
Gender,,
Female,17.45,100
Male,81.15,465
Other / Non-Disclosed,1.40,8


In [5]:
#Purchasing Analysis (Gender) 
pd.options.display.float_format = '${:,.2f}'.format

#The below each broken by gender

#Purchase Count
df["Gender"].value_counts()

#Average Purchase Price
g1=df.groupby(['Gender']).mean()
g1.Price

#Total Purchase Value
g2=df.groupby(['Gender']).sum()
g2.Price

#Normalized Totals
g3=df.groupby(['Gender']).sum()/player_count.count()
g3.Price


df_purchase_analysis_gen = pd.DataFrame({
                   'Purchase Count': df["Gender"].value_counts(),
                   'Average Purchase Price': g1.Price,
                   'Total Purchase Value': g2.Price,
                   'Normalized Totals': g3.Price
                             })
df_purchase_analysis_gen.index.name = 'Gender'

df_purchase_analysis_gen
df_purchase_analysis_gen[["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$0.67
Male,633,$2.95,"$1,867.68",$3.26
Other / Non-Disclosed,11,$3.25,$35.74,$0.06


In [6]:
#Age Demographics
pd.options.display.float_format = '{:,.2f}'.format

df_age=pd.DataFrame({'count' : df.groupby( [ "SN", "Age"] ).size()}).reset_index()


#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.) 
# Create the bins in which Data will be held
# Bins are <10, 10-14, 15-19, 20-24, 25-29, 30-34



bins = [0, 10, 15, 20, 25, 30, 35, 39, 43]

# Create the names for the four bins
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44']

# Cut Age and place the players into bins
pd.cut(df_age["Age"], bins, labels=group_names)

df_age["Age Group"] = pd.cut(df_age["Age"], bins, labels=group_names)
df.head()

#count by bin
count_by_bin=df_age.groupby(['Age Group']).size()
#count_by_bin

count_by_bin_percent=df_age.groupby(['Age Group']).size()/player_count.count()*100
#count_by_bin_percent
df_age_demo = pd.DataFrame({ 
                   'Percentage of Players': count_by_bin_percent,
                   'Total Count': count_by_bin
                             })
df_age_demo

,Percentage of Players,Total Count
Age Group,,
<10,3.84,22
10-14,9.42,54
15-19,24.26,139
20-24,40.84,234
25-29,9.08,52
30-34,7.68,44
35-39,2.97,17
40-44,1.75,10


In [7]:
#Purchasing Analysis (Age)
pd.options.display.float_format = '${:,.2f}'.format

# Bins are <10, 10-14, 15-19, 20-24, 25-29, 30-34
bins = [0, 10, 15, 20, 25, 30, 35, 39, 43]

# Create the names for the bins
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44']

# Cut Age and place the players into bins
pd.cut(df["Age"], bins, labels=group_names)

df["Age Group"] = pd.cut(df["Age"], bins, labels=group_names)
df.head()

#Purchase Count
purchase_count_by_age=df.groupby(['Age Group']).size()


#Average Purchase Price
age_groupby_mean = df.groupby(['Age Group']).mean()
age_groupby_mean.Price

#Total Purchase Value
age_groupby_sum = df.groupby(['Age Group']).sum()
age_groupby_sum.Price

#Normalized Totals
normalized_player_total = df.groupby(['Age Group']).sum()
normalized_player_total.Price/player_count.count()

df_purchasing_analysis_by_age = pd.DataFrame({
                   'Purchase Count': purchase_count_by_age,
                   'Average Purchase Price': age_groupby_mean.Price,
                   'Total Purchase Value': age_groupby_sum.Price,
                   'Normalized Totals': normalized_player_total.Price/player_count.count()
                             })
df_purchasing_analysis_by_age
df_purchasing_analysis_by_age[["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,32,$3.02,$96.62,$0.17
10-14,78,$2.87,$224.15,$0.39
15-19,184,$2.87,$528.74,$0.92
20-24,305,$2.96,$902.61,$1.58
25-29,76,$2.89,$219.82,$0.38
30-34,58,$3.07,$178.26,$0.31
35-39,30,$2.75,$82.38,$0.14
40-44,16,$3.19,$51.03,$0.09


In [8]:
#Top Spenders
pd.options.display.float_format = '${:,.2f}'.format

#setting up output data frame
top_spender_df = pd.DataFrame({
     "SN": df['SN'],
     "Price": df['Price'],
     "Item ID": df['Item ID']})

top_spender_df


#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):


#total items bought by each user
top_spender_groupby = top_spender_df.groupby(['SN']).count()
top_spender_groupby

#total purchase price
top_spender_price_groupby1 = top_spender_df.groupby(['SN']).sum()
top_spender_price_groupby1

#getting total purchase value
top_spender_groupby["Price"] = top_spender_price_groupby1["Price"]

#average purchase price
top_spender_groupby["Average Purchase Price"] = (top_spender_groupby["Price"]/top_spender_groupby["Item ID"])
top_spender_groupby

#renaming columns to match hw
top_spender_groupby = top_spender_groupby.rename(columns={'Price': 'Total Purchase Value', 
                                                          'Item ID': 'Purchase Count',
                                                          })

top_spender_groupby

top_spender_sorted = top_spender_groupby.sort_values('Total Purchase Value', ascending=False)
top_spender_sorted

top5=top_spender_sorted.head(5)
top5

top5[["Purchase Count","Average Purchase Price","Total Purchase Value"]]
#SN
#Purchase Count
#Average Purchase Price
#Total Purchase Value

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [9]:
#Most Popular Items
pd.options.display.float_format = '${:,.2f}'.format

#setting up output data frame

most_popular_df = pd.DataFrame({
     "Item ID": df['Item ID'],
     "Item Name": df['Item Name'],
     "Price": df['Price']
  })

most_popular_df


#Identify the 5 most popular items by purchase count, then list (in a table):

top_items_groupby = most_popular_df.groupby(['Item Name','Item ID']).count()
top_items_groupby["Purchase Count"]=top_items_groupby["Price"]
top_items_total_purchase_price_groupby = most_popular_df.groupby(['Item Name','Item ID']).sum()
top_items_groupby["Price"]=top_items_total_purchase_price_groupby["Price"]
top_items_groupby["Item Price"]=(top_items_total_purchase_price_groupby["Price"]/top_items_groupby["Purchase Count"])


#renaming columns to match hw
top_item = top_items_groupby.rename(columns={'Price': 'Total Purchase Value' 
                                                          })
top_item.head()

top_item_sorted = top_item.sort_values('Purchase Count', ascending=False)

top5=top_item_sorted.head(5)

top5=top5.reset_index()
top5
df_top = pd.DataFrame({
                   'Item ID': top5["Item ID"],
                   'Item Name': top5["Item Name"],
                   'Purchase Count': top5["Purchase Count"],
                   'Item Price': top5["Item Price"],
                   'Total Purchase Value': top5["Total Purchase Value"]
          }).set_index('Item ID')

df_top

,Item Name,Item Price,Purchase Count,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
84,Arcane Gem,$2.23,11,$24.53
34,Retribution Axe,$4.14,9,$37.26
31,Trickster,$2.07,9,$18.63
13,Serenity,$1.49,9,$13.41


In [10]:
#Most Profitable Items
pd.options.display.float_format = '${:,.2f}'.format

#Identify the 5 most profitable items by total purchase value, then list (in a table):


#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value

most_profit_df = pd.DataFrame({
     "Item ID": df['Item ID'],
     "Item Name": df['Item Name'],
     "Price": df['Price']
})

most_profit_df

#items by purchase value
pop_items_groupby = most_profit_df.groupby(['Item Name','Item ID']).sum()

#items by purchase count
pop_items_groupby["Purchase Count"] = most_profit_df.groupby(['Item Name','Item ID']).count()

#item price
pop_items_groupby["Item Price"] = most_profit_df.groupby(['Item Name','Item ID']).sum()/most_profit_df.groupby(['Item Name','Item ID']).count()

#renaming columns to match hw
pop_items_groupby = pop_items_groupby.rename(columns={'Price': 'Total Purchase Value' 
                                                     })
#sort by total purchase value
pop_item_sorted = pop_items_groupby.sort_values('Total Purchase Value', ascending=False)

pop5=pop_item_sorted.head(5)
pop5=pop5.reset_index()

df_pop = pd.DataFrame({
                   'Item ID': pop5["Item ID"],
                   'Item Name': pop5["Item Name"],
                   'Purchase Count': pop5["Purchase Count"],
                   'Item Price': pop5["Item Price"],
                   'Total Purchase Value': pop5["Total Purchase Value"]
                             }).set_index('Item ID')

df_pop


,Item Name,Item Price,Purchase Count,Total Purchase Value
Item ID,,,,
34,Retribution Axe,$4.14,9,$37.26
115,Spectral Diamond Doomblade,$4.25,7,$29.75
32,Orenmir,$4.95,6,$29.70
103,Singed Scalpel,$4.87,6,$29.22
107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88
